In [28]:
# TO DO: My conditions tracker doesn't work. I need to fix the columns by changing the delimiter. Then, I need to make sure that my 'merge' function is actually doing what I want it to.
# Then make the plots. 

In [2]:
#@title Dependencies

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import pprint
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import numpy as np

In [3]:
# Load in the files
df = pd.read_csv("C:\\Users\\alexa\\OneDrive\\Documents\\MSc Psychology\\MSc Thesis\\Code\\experiment_results_v3.csv") # Experiment results, wrangled in R
conditions_tracker = pd.read_csv("C:\\Users\\alexa\\OneDrive\\Documents\\MSc Psychology\\MSc Thesis\\Code\\experiment_conditions_tracking_semicolon.csv", delimiter=';')

# Manually check that columns match.
# df and conditions_tracker both need: 'sentence_id', 'probe_type', 'probe_number'
# and that conditions_tracker has 'interval_length', 'complexity', 'concept_tag'


In [4]:
print(df)

     Unnamed: 0         ResponseId  attention  \
0             1  R_2E70Gw66U1MFTey          6   
1             2  R_2E70Gw66U1MFTey          6   
2             3  R_2E70Gw66U1MFTey          6   
3             4  R_2E70Gw66U1MFTey          6   
4             5  R_2E70Gw66U1MFTey          6   
..          ...                ...        ...   
944         945  R_8ZP9f4t8l9cLCIg          7   
945         946  R_8ZP9f4t8l9cLCIg          7   
946         947  R_8ZP9f4t8l9cLCIg          7   
947         948  R_8ZP9f4t8l9cLCIg          7   
948         949  R_8ZP9f4t8l9cLCIg          7   

                        sentence_name  \
0               abovetruthvaluequery2   
1                 agetruthvaluequery1   
2          directionstruthvaluequery2   
3               goingtruthvaluequery1   
4            surprisetruthvaluequery1   
..                                ...   
944          namescontinuationprompt1   
945  continuation3continuationprompt1   
946            newcontinuationprompt2   
9

In [5]:
# Define a function to parse the question column
def parse_question(sentence_name):
    match = re.match(r'([a-zA-Z0-9_]+)(truthvaluequery|continuationprompt)(\d+)', sentence_name)
    if match:
        sentence_id, probe_type, probe_number = match.groups()
        return pd.Series([sentence_id, probe_type, int(probe_number)])
    return pd.Series([sentence_name, None, None])  # Return question for manual inspection

In [6]:
# Apply the function to the question column and create new columns
# This is so that we can match the columns to the conditions tracker in order to add the appropriate conditions
df[['sentence_id', 'probe_type', 'probe_number']] = df['sentence_name'].apply(parse_question)

In [7]:
# Inspect the data

print(df)

     Unnamed: 0         ResponseId  attention  \
0             1  R_2E70Gw66U1MFTey          6   
1             2  R_2E70Gw66U1MFTey          6   
2             3  R_2E70Gw66U1MFTey          6   
3             4  R_2E70Gw66U1MFTey          6   
4             5  R_2E70Gw66U1MFTey          6   
..          ...                ...        ...   
944         945  R_8ZP9f4t8l9cLCIg          7   
945         946  R_8ZP9f4t8l9cLCIg          7   
946         947  R_8ZP9f4t8l9cLCIg          7   
947         948  R_8ZP9f4t8l9cLCIg          7   
948         949  R_8ZP9f4t8l9cLCIg          7   

                        sentence_name  \
0               abovetruthvaluequery2   
1                 agetruthvaluequery1   
2          directionstruthvaluequery2   
3               goingtruthvaluequery1   
4            surprisetruthvaluequery1   
..                                ...   
944          namescontinuationprompt1   
945  continuation3continuationprompt1   
946            newcontinuationprompt2   
9

In [8]:
# Filter rows where probe_type and probe_number are None (indicating no match)
unmatched_sentences = df[df['score'].isna()]

# Display the result
print(unmatched_sentences)

# Note: returns empty df, meaning parsing the question column worked

Empty DataFrame
Columns: [Unnamed: 0, ResponseId, attention, sentence_name, answer, survey, row_id, score, sentence_id, probe_type, probe_number]
Index: []


In [9]:
# Rename probe_type values, this matches the conditions tracker
df['probe_type'] = df['probe_type'].replace({'truthvaluequery': 'truth_value', 'continuationprompt': 'continuation'})

In [10]:
# Check that the column names match
# We need 'survey', 'probe_type', and 'sentence_id' to match

print(df.columns)
print(conditions_tracker.columns)

Index(['Unnamed: 0', 'ResponseId', 'attention', 'sentence_name', 'answer',
       'survey', 'row_id', 'score', 'sentence_id', 'probe_type',
       'probe_number'],
      dtype='object')
Index(['survey', 'sentence_id', 'interval_length', 'probe_type', 'version',
       'complexity', 'concept_tag'],
      dtype='object')


In [11]:
# Convert any potential mismatches in data type

df['survey'] = df['survey'].astype(str)
df['sentence_id'] = df['sentence_id'].astype(str)
df['probe_type'] = df['probe_type'].astype(str)

conditions_tracker['survey'] = conditions_tracker['survey'].astype(str)
conditions_tracker['sentence_id'] = conditions_tracker['sentence_id'].astype(str)
conditions_tracker['probe_type'] = conditions_tracker['probe_type'].astype(str)


In [12]:
# Now we are going to add the conditions: interval length, complexity, and concept tag

merged_df = pd.merge(df, conditions_tracker, on=['survey', 'sentence_id', 'probe_type'], how='left')


In [14]:
print(df['sentence_id'].unique())
print(conditions_tracker['sentence_id'].unique()) # extra elements: 'injury', 'emotions', 'emotions_(self)'


['above' 'age' 'directions' 'going' 'surprise' 'largest' 'shadow'
 'surrounding' 'alive' 'ToM' 'eating' 'coming' 'taste2' 'color' 'far'
 'more' 'completion' 'empty' 'smallest' 'continuation3' 'cutting'
 'reciprocity' 'speed' 'age2' 'support' 'efficiency' 'fill' 'dying'
 'touch' 'day' 'largest2' 'correspondence' 'matter' 'names' 'slower'
 'goaldirectedness' 'dry' 'besides' 'copy' 'new' 'smallest2'
 'continuation2']
['eating' 'coming' 'above' 'age' 'directions' 'going' 'taste2' 'surprise'
 'color' 'far' 'more' 'completion' 'largest' 'empty' 'shadow' 'injury_'
 'surrounding' 'smallest' 'continuation3' 'alive' 'cutting' 'reciprocity'
 'speed' 'ToM' 'touch' 'day' 'age2' 'support' 'largest2' 'efficiency'
 'correspondence' 'matter' 'names' 'fill' 'dying' 'emotions_(self)' 'copy'
 'new' 'smallest2' 'slower' 'goaldirectedness' 'dry' 'besides'
 'continuation2' 'injury' 'emotions']


In [13]:
# Assign the conditions to the matching rows

df['interval_length'] = merged_df['interval_length']
df['complexity'] = merged_df['complexity']
df['concept_tag'] = merged_df['concept_tag']

In [14]:
# Check if there's any missing values after merging

missing_data_rows = merged_df[merged_df.isnull().any(axis=1)]
print(missing_data_rows)

# Yay it's empty!

Empty DataFrame
Columns: [Unnamed: 0, ResponseId, attention, sentence_name, answer, survey, row_id, score, sentence_id, probe_type, probe_number, interval_length, version, complexity, concept_tag]
Index: []


In [15]:
print(merged_df)

     Unnamed: 0         ResponseId  attention  \
0             1  R_2E70Gw66U1MFTey          6   
1             2  R_2E70Gw66U1MFTey          6   
2             3  R_2E70Gw66U1MFTey          6   
3             4  R_2E70Gw66U1MFTey          6   
4             5  R_2E70Gw66U1MFTey          6   
..          ...                ...        ...   
944         945  R_8ZP9f4t8l9cLCIg          7   
945         946  R_8ZP9f4t8l9cLCIg          7   
946         947  R_8ZP9f4t8l9cLCIg          7   
947         948  R_8ZP9f4t8l9cLCIg          7   
948         949  R_8ZP9f4t8l9cLCIg          7   

                        sentence_name  \
0               abovetruthvaluequery2   
1                 agetruthvaluequery1   
2          directionstruthvaluequery2   
3               goingtruthvaluequery1   
4            surprisetruthvaluequery1   
..                                ...   
944          namescontinuationprompt1   
945  continuation3continuationprompt1   
946            newcontinuationprompt2   
9

In [16]:
# Split truth value and continuation prompts out 

truth_value_df = merged_df[merged_df['probe_type'] == 'truth_value']
continuation_df = merged_df[merged_df['probe_type'] == 'continuation']

In [17]:
print(truth_value_df.columns)

Index(['Unnamed: 0', 'ResponseId', 'attention', 'sentence_name', 'answer',
       'survey', 'row_id', 'score', 'sentence_id', 'probe_type',
       'probe_number', 'interval_length', 'version', 'complexity',
       'concept_tag'],
      dtype='object')


In [78]:
# Save locally

truth_value_df.to_csv('human_truth_value_results_14_11.csv', index=False)
continuation_df.to_csv('human_continuation_results_14_11.csv', index=False)